This file is meant as a helper file for the project. 
After turning pgn into .csv file, we save it, then we will load it alongside the kaggle dataset and combine the two into one final dataset to feed the chess app.
Columns in the kaggle dataset: </br>
id, created_at, last_move_at, turns, victory_status, winner, increment_code, white_rating, black_rating, moves, opening_eco, opening_name, opening_ply, endFEN, mated_by, nr_of_pieces, pawns, knights, bishops, rooks, queens, wKing_sqr, bKing_sqr

In [1]:
import dtale
import pandas as pd
import numpy as np

import chess
import chess.pgn


In [2]:
# Defines dict to map notation of pieces to their respective string representation
pieces_san_dict = {"P":"Pawn","N":"Knight","B":"Bishop","R":"Rook","Q":"Queen","K":"King"}

# Define function that returns which piece delivered mate, by parsing the moves column.
def mate_piece(move_list):
    last_move = move_list[-1]
    if "#" in last_move:
        #consider that pawns don't have uppecase lettering. 
        if last_move[0].isupper():
            return pieces_san_dict[last_move[0]]
        else:
            return 'Pawn'
    else:
        return None

In [3]:
# Read PGN files.
# pgn_file = "lichess_elite_2014-05.pgn" 
pgn_file = "lichess_db_standard_rated_2017-04.pgn"

In [4]:
# First, skim through the pgn to extract data from the headers.

# Columns to extract.
extracting = ["Event","UTCDate","Result","WhiteElo","BlackElo","Termination","TimeControl","ECO","Opening"]

# Initialize a dict that will be the base for the output DF.
df_dict = {k: [] for k in extracting+["fens","moves","turns","mated_by"]}

# Extract headers.
# Note that read_headers actually reads the file in order, so to start over we need to open it again.
pgn = open(pgn_file)

iterations = 0
while iterations<500000:
        
    headers = chess.pgn.read_headers(pgn)
    if headers is None:
        break
        
    for header in extracting:
        df_dict[header].append(headers[header])
    iterations += 1

print("Total number of games' headers parsed: {}".format(iterations))
amt_games = iterations
    
# Extract the other columns.
# This defines the read_game argument to read all the way to the end of the game and return the final position of the board, which is what we want.
pgn = open(pgn_file)
board_builder = chess.pgn.BoardBuilder

iterations = 0
while iterations < amt_games:
    finalboard = chess.pgn.read_game(pgn,Visitor=board_builder)
    df_dict["moves"].append(finalboard.fullmove_number)
    df_dict["turns"].append(finalboard.ply())
    if finalboard.is_checkmate():
        df_dict["mated_by"].append(pieces_san_dict[finalboard.piece_at(finalboard.peek().to_square).symbol().upper()])
    else:
        df_dict["mated_by"].append(None)
    df_dict["fens"].append(finalboard.fen())

    iterations += 1

print("Total number of games parsed: {}".format(iterations))


#print(df_dict)

Total number of games' headers parsed: 500000
Total number of games parsed: 500000


In [5]:
# Turn df_dict into a dataframe, that matches the one on Kaggle Dataset.
df = pd.DataFrame(df_dict)

In [6]:
df.head(5)

,Event,UTCDate,Result,WhiteElo,BlackElo,Termination,TimeControl,ECO,Opening,fens,moves,turns,mated_by
0,Rated Blitz game,2017.03.31,1-0,2186,1907,Normal,180+0,C34,"King's Gambit Accepted, Schallopp Defense",r1b2q1b/pp1pkB1r/2pN4/4P1Q1/1n1P3P/2N5/PPPK1RP...,19,37,None
1,Rated Bullet game,2017.03.31,1-0,1385,1339,Time forfeit,120+1,C34,"King's Gambit Accepted, King's Knight Gambit",6k1/pp1b1p1p/1b1B1Q2/3B4/3P3N/2P3K1/PP4PP/r7 b...,24,47,None
2,Rated Blitz game,2017.03.31,1-0,1905,1836,Normal,180+0,C63,"Ruy Lopez: Schliemann Defense, Schoenemann Attack",4B2r/6b1/5p1k/2p4Q/4pP2/8/PPP3RK/8 b - - 0 33,33,65,Queen
3,Rated Bullet game,2017.03.31,0-1,1670,1812,Normal,120+1,A00,Hungarian Opening,8/1kpp1N2/4p1n1/1p2Pp2/2n2Pp1/P3K1P1/8/8 w - -...,38,74,None
4,Rated Blitz game,2017.03.31,0-1,988,1145,Time forfeit,180+0,A00,Van't Kruijs Opening,r3k3/1p3pp1/p1p5/3pN3/1P1P4/P1NP4/1R6/2K2q2 w ...,32,62,None


In [7]:
df.shape

(500000, 13)

In [8]:
df_bk = df.copy()
#df = df_bk.copy()

In [9]:
# Manipulate columns to resemble the kaggle dataset.
# Drop some columns.
#df.drop(columns="Date",inplace=True)

## Fix dtypes.
df["WhiteElo"] = df["WhiteElo"].astype(int)
df["BlackElo"] = df["BlackElo"].astype(int)

In [10]:
# Result -> Winner
replace_dict = {"1-0":"white","0-1":"black",'1/2-1/2':"draw"}
df["Winner"] = df["Result"].replace(to_replace=replace_dict)
df.drop(columns="Result",inplace=True)

In [11]:
# Termination -> Victory Status. In kaggle theres outoftime, resign, draw, and mate.
## index of different conditions that will make a new column.
draw_idx = np.where(df["Winner"]=="draw")
time_idx = np.where(df["Termination"]=="Time forfeit")
mate_idx = np.where(df["mated_by"].isna()==False)

## The rest will be filled with resign, as there is no other option left.
df["victory_status"] = "resign"
df["victory_status"].iloc[draw_idx[0]] = "draw"
df["victory_status"].iloc[time_idx[0]] = "outoftime"
df["victory_status"].iloc[mate_idx[0]] = "mate"
try:
    df.drop(columns="Termination",inplace=True)
except:
    pass
## who cares if this raises warnings, warnings are not errors.

C:\Users\Tiago\.conda\envs\JupyterLab v2x\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
# Apply the same functions as to Kaggle dataset to derive the remaining columns.

# Code in this cell is adapted from https://andreasstckl.medium.com/chessviz-graphs-of-chess-games-7ebd4f85a9b9
# Define a function that coverts the FEN notation into a tensor representation of the board.

def fen_to_tensor(input_str):
    # creates a dictionary of pieces. 1 to 6 is white pieces, -1 to -6 is black pieces.
    pieces_str = "PNBRQKpnbrqk"
    pieces = set(pieces_str)
    pieces_dict = {pieces_str[0]:1, pieces_str[1]:2, pieces_str[2]:3, pieces_str[3]:4 , 
                    pieces_str[4]:5, pieces_str[5]:6,
                    pieces_str[6]:-1, pieces_str[7]:-2, pieces_str[8]:-3, pieces_str[9]:-4, 
                    pieces_str[10]:-5, pieces_str[11]:-6}
    
    # creates a tensor representation of the board. 
    # This tensor represents a 8*8 board for each of the 6 possible pieces.
    # The first two axis are the coordinates, the third axis is whether there is a piece there or not.
    # If 1, there is a white piece there. If -1, there is a black piece there.
    # For example, if the value at position (0,0,0) is 1, then there is a white pawn at the A8 sqr.
    board_tensor = np.zeros((8,8,6))
    input_list = input_str.split()
    rownr = 0
    colnr = 0
    valid_spaces = set(range(1,9))
    
    # As input list is a split FEN, index 0 is the pos of each piece on the board.
    # This loop goes through every element of the FEN which is the same as looking at each square of the board in sequence and seeing what piece is there.
    for i,c in enumerate(input_list[0]):
        if c in pieces:
            board_tensor[rownr,colnr,np.abs(pieces_dict[c])-1] = np.sign(pieces_dict[c])
            colnr = colnr + 1
        elif c == "/": 
            #a backslash means the row is over, so we go to the next row
            rownr += 1
            colnr = 0
        elif int(c) in valid_spaces:
            # a number means that many squares in the row are empty.
            colnr += int(c)
        else:
            raise ValueError("invalid fenstr at index: {} char: {}".format(i, c))
        
    return board_tensor

# Define other useful functions for these tensors.
# Counts the total amount of pieces on the board.
def count_pieces(fen):
    board_tensor = fen_to_tensor(fen)
    return np.sum(np.abs(board_tensor))

# This outputs a tuple containing the total amt of pawns, knights, bishops, rooks, and queens in the board.
def count_all_pieces(fen):
    board_tensor = fen_to_tensor(fen)
    counts = np.sum(np.abs(board_tensor),axis=(0,1))
    return counts[0],counts[1],counts[2],counts[3],counts[4]

# This outputs both King's square position
def king_squares(fen):
    board_tensor = fen_to_tensor(fen)
    king_col,king_row=np.where(board_tensor[:,:,5]==1)
    white_king_coordinates = king_col[0],king_row[0]
    king_col,king_row=np.where(board_tensor[:,:,5]==-1)
    black_king_coordinates = king_col[0],king_row[0]
    
    return white_king_coordinates,black_king_coordinates

def queen_squares(fen):
    board_tensor = fen_to_tensor(fen)
    try:
        queen_col,queen_row=np.where(board_tensor[:,:,4]==1)
        white_queen_coordinates = queen_col[0],queen_row[0]
    except:
        white_queen_coordinates = (None,None)
        
    try:
        queen_col,queen_row=np.where(board_tensor[:,:,4]==-1)
        black_queen_coordinates = queen_col[0],queen_row[0]
    except:
        black_queen_coordinates = (None,None)
        
    
    return white_queen_coordinates,black_queen_coordinates

def rook_squares(fen):
    board_tensor = fen_to_tensor(fen)
    try:
        rook_col,rook_row=np.where(board_tensor[:,:,3]==1)
        white_rook_coordinates = rook_col[0],rook_row[0]
    except:
        white_rook_coordinates = (None,None)
    try:
        white_rook2_coordinates = rook_col[1],rook_row[1]
    except:
        white_rook2_coordinates = (None,None)
        
    try:
        rook_col,rook_row=np.where(board_tensor[:,:,3]==-1)
        black_rook_coordinates = rook_col[0],rook_row[0]
    except:
        black_rook_coordinates = (None,None)
    try:
        black_rook2_coordinates = rook_col[1],rook_row[1]
    except:
        black_rook2_coordinates = (None,None)
        
    
    return white_rook_coordinates,black_rook_coordinates,white_rook2_coordinates,black_rook2_coordinates

def bishop_squares(fen):
    board_tensor = fen_to_tensor(fen)
    try:
        bishop_col,bishop_row=np.where(board_tensor[:,:,2]==1)
        white_bishop_coordinates = bishop_col[0],bishop_row[0]
    except:
        white_bishop_coordinates = (None,None)
    try:
        white_bishop2_coordinates = bishop_col[1],bishop_row[1]
    except:
        white_bishop2_coordinates = (None,None)
        
    try:
        bishop_col,bishop_row=np.where(board_tensor[:,:,2]==-1)
        black_bishop_coordinates = bishop_col[0],bishop_row[0]
    except:
        black_bishop_coordinates = (None,None)
    try:
        black_bishop2_coordinates = bishop_col[1],bishop_row[1]
    except:
        black_bishop2_coordinates = (None,None)
        
    
    return white_bishop_coordinates,black_bishop_coordinates,white_bishop2_coordinates,black_bishop2_coordinates

def knight_squares(fen):
    board_tensor = fen_to_tensor(fen)
    try:
        knight_col,knight_row=np.where(board_tensor[:,:,1]==1)
        white_knight_coordinates = knight_col[0],knight_row[0]
    except:
        white_knight_coordinates = (None,None)
    try:
        white_knight2_coordinates = knight_col[1],knight_row[1]
    except:
        white_knight2_coordinates = (None,None)
        
    try:
        knight_col,knight_row=np.where(board_tensor[:,:,1]==-1)
        black_knight_coordinates = knight_col[0],knight_row[0]
    except:
        black_knight_coordinates = (None,None)
    try:
        black_knight2_coordinates = knight_col[1],knight_row[1]
    except:
        black_knight2_coordinates = (None,None)
        
    
    return white_knight_coordinates,black_knight_coordinates,white_knight2_coordinates,black_knight2_coordinates

In [13]:
## Now let's use the FENS and functions we have to derive more columns.

df.rename(columns={"fens":"endFEN"},inplace=True)
# Total amount of pieces on the board.
df["nr_of_pieces"] = df["endFEN"].apply(count_pieces)

# gets the nr of each piece on the board
temp = df["endFEN"].apply(count_all_pieces)
for i,piece in enumerate(["pawns","knights","bishops","rooks","queens"]):
    df[piece] = temp.apply(lambda x:x[i])
# This might not be very useful because it doesn't discern between white and black pieces, but that is okay for now. 
# It can be used for filters based on presence of pieces at endgame.
# We can use the number of pieces and turns to define arbitrarily if the game ended in the opening, midgame, or endgame.

# Defines columns for both kings positions
temp = df["endFEN"].apply(king_squares)
for i,sqr in enumerate(["wKing_sqr","bKing_sqr"]):
    df[sqr] = temp.apply(lambda x:x[i])

# Defines columns for both queens positions
temp = df["endFEN"].apply(queen_squares)
for i,sqr in enumerate(["wQueen_sqr","bQueen_sqr"]):
    df[sqr] = temp.apply(lambda x:x[i])

# Defines columns for all rook positions
temp = df["endFEN"].apply(rook_squares)
for i,sqr in enumerate(["wRook_sqr","bRook_sqr","wRook2_sqr","bRook2_sqr"]):
    df[sqr] = temp.apply(lambda x:x[i])
    
# Defines columns for all bishop positions
temp = df["endFEN"].apply(bishop_squares)
for i,sqr in enumerate(["wBishop_sqr","bBishop_sqr","wBishop2_sqr","bBishop2_sqr"]):
    df[sqr] = temp.apply(lambda x:x[i])
    
# Defines columns for all knight positions
temp = df["endFEN"].apply(knight_squares)
for i,sqr in enumerate(["wKnight_sqr","bKnight_sqr","wKnight2_sqr","bKnight2_sqr"]):
    df[sqr] = temp.apply(lambda x:x[i])

In [14]:
## Derive more columns if necessary.
df["avg_Elo"] = df.loc[:,"WhiteElo":"BlackElo"].mean(axis=1)

In [15]:
df.head(10)

,Event,UTCDate,WhiteElo,BlackElo,TimeControl,ECO,Opening,endFEN,moves,turns,...,bRook2_sqr,wBishop_sqr,bBishop_sqr,wBishop2_sqr,bBishop2_sqr,wKnight_sqr,bKnight_sqr,wKnight2_sqr,bKnight2_sqr,avg_Elo
0,Rated Blitz game,2017.03.31,2186,1907,180+0,C34,"King's Gambit Accepted, Schallopp Defense",r1b2q1b/pp1pkB1r/2pN4/4P1Q1/1n1P3P/2N5/PPPK1RP...,19,37,...,"(1, 7)","(1, 5)","(0, 2)","(None, None)","(0, 7)","(2, 3)","(4, 1)","(5, 2)","(None, None)",2046.5
1,Rated Bullet game,2017.03.31,1385,1339,120+1,C34,"King's Gambit Accepted, King's Knight Gambit",6k1/pp1b1p1p/1b1B1Q2/3B4/3P3N/2P3K1/PP4PP/r7 b...,24,47,...,"(None, None)","(2, 3)","(1, 3)","(3, 3)","(2, 1)","(4, 7)","(None, None)","(None, None)","(None, None)",1362.0
2,Rated Blitz game,2017.03.31,1905,1836,180+0,C63,"Ruy Lopez: Schliemann Defense, Schoenemann Attack",4B2r/6b1/5p1k/2p4Q/4pP2/8/PPP3RK/8 b - - 0 33,33,65,...,"(None, None)","(0, 4)","(1, 6)","(None, None)","(None, None)","(None, None)","(None, None)","(None, None)","(None, None)",1870.5
3,Rated Bullet game,2017.03.31,1670,1812,120+1,A00,Hungarian Opening,8/1kpp1N2/4p1n1/1p2Pp2/2n2Pp1/P3K1P1/8/8 w - -...,38,74,...,"(None, None)","(None, None)","(None, None)","(None, None)","(None, None)","(1, 5)","(2, 6)","(None, None)","(4, 2)",1741.0
4,Rated Blitz game,2017.03.31,988,1145,180+0,A00,Van't Kruijs Opening,r3k3/1p3pp1/p1p5/3pN3/1P1P4/P1NP4/1R6/2K2q2 w ...,32,62,...,"(None, None)","(None, None)","(None, None)","(None, None)","(None, None)","(3, 4)","(None, None)","(5, 2)","(None, None)",1066.5
5,Rated Blitz game,2017.03.31,1367,1396,180+0,A00,Hungarian Opening,r4rk1/ppp1q1pp/2p2pn1/8/8/1P2P1Pb/PbPNNP1P/1R1...,14,27,...,"(0, 5)","(None, None)","(5, 7)","(None, None)","(6, 1)","(6, 3)","(2, 6)","(6, 4)","(None, None)",1381.5
6,Rated Blitz game,2017.03.31,1683,1832,180+0,C00,French Defense: Knight Variation,8/8/8/5k2/3K4/6p1/1P2B3/8 b - - 0 51,51,101,...,"(None, None)","(6, 4)","(None, None)","(None, None)","(None, None)","(None, None)","(None, None)","(None, None)","(None, None)",1757.5
7,Rated Bullet game,2017.03.31,1515,1576,120+0,B10,Caro-Kann Defense: Accelerated Panov Attack #2,8/5p1k/7P/8/4pK2/8/pb3P2/8 w - - 0 49,49,96,...,"(None, None)","(None, None)","(6, 1)","(None, None)","(None, None)","(None, None)","(None, None)","(None, None)","(None, None)",1545.5
8,Rated Classical game,2017.03.31,1822,1885,300+8,C64,"Ruy Lopez: Classical Variation, Central Variation",4r3/6pk/R2p3p/2p5/5N2/1PB2N1P/1P2r1PK/8 b - - ...,31,61,...,"(6, 4)","(5, 2)","(None, None)","(None, None)","(None, None)","(4, 5)","(None, None)","(5, 5)","(None, None)",1853.5
9,Rated Classical tournament https://lichess.org...,2017.03.31,1766,1768,600+0,C16,"French Defense: Winawer Variation, Advance Var...",8/6k1/7p/2Q2pn1/3Pq3/6PK/5P1P/8 w - - 7 42,42,82,...,"(None, None)","(None, None)","(None, None)","(None, None)","(None, None)","(None, None)","(3, 6)","(None, None)","(None, None)",1767.0


### Dataset Export

In [16]:
# Export the preprocessed dataset.
df_export = df.copy()
df_export.to_csv("chess_app.csv")